In [21]:
import numpy as np
from numpy import exp,array,random,dot
import pandas as pd

def sigmoid(x):
    return 1/(1+np.exp(-x))

def derv_sigmoid(x):
    return x*(1-x)
    

In [10]:
train_set_ip = array([[0,0,1],[1,1,1],[1,0,1],[0,1,1]])
train_set_op = array([[0,1,1,0]]).T

random.seed(1)

weights = 2 * random.random((3,1)) - 1

for i in range(1000):
    op = 1 / (1 + exp(-dot(train_set_ip,weights)))
    weights = weights + dot(train_set_ip.T,(train_set_op - op) * op * (1-op))
res = 1 / (1 + exp(-dot(array([1,0,0]),weights))) 
print(res)

[ 0.99929937]


In [46]:
data = pd.read_csv("data/train.csv")

In [59]:
#Remove rows where workclass is unknown
data.drop(data["workclass"] == " ?")
data[data.age != 25]
data.head()

data.to_csv("data/data1.csv", sep=',',header=True,index=False)

In [34]:
#Classifier data entries to columns
data = pd.get_dummies(data)
data.to_csv("data/data_afterdum.csv", sep=',',header=True,index=False)